In [1]:
import cv2
#from tracker import *
import torch
import numpy as np

## 1. Load YOLO Object Detector

In [2]:
# Cargar el modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Definir clases de interés
CLASSES_OF_INTEREST = ['car']

Using cache found in C:\Users\Josep/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-9-24 Python-3.11.9 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


## 2. Define Tracker Class (per centroides)

In [4]:
import numpy as np
from scipy.spatial.distance import cdist

class Tracker:
    MAX_DISAPPEAR_LIMIT = 3
    def __init__(self):
        self.next_unique_id = 0
        self.trackers = {}
        self.disappear_trackers = {}
        self.tracked_bboxes = {}
    
    
    def init_object(self,centroid,boxes):
        global next_unique_id
        self.trackers[self.next_unique_id] = centroid
        self.tracked_bboxes[self.next_unique_id] = boxes
        self.disappear_trackers[self.next_unique_id] = 0
        self.next_unique_id+=1

    def del_object(self,track_id):
        del self.trackers[track_id]
        del self.tracked_bboxes[track_id]
        del self.disappear_trackers[track_id]

    def update_object(self,bboxes):
        
        if(len(bboxes)==0):
            
            for oid in list(self.disappear_trackers.keys()):
                self.disappear_trackers[oid]+=1
                
                if self.disappear_trackers[oid] > Tracker.MAX_DISAPPEAR_LIMIT:
                    self.del_object(oid)
                
            return self.tracked_bboxes
        
        else:   
            input_centroids = np.zeros((len(bboxes),2)) 
            for i in range(len(bboxes)):
                x,y,w,h = bboxes[i][0],bboxes[i][1],bboxes[i][2],bboxes[i][3]
                cx,cy = x + w/2 , y + h/2
                input_centroids[i] = (cx,cy)

            
            if(len(self.trackers)==0):
                for i in range(len(input_centroids)):
                    self.init_object(input_centroids[i],bboxes[i])
            
            else:
                
                tracker_centroids = list(self.trackers.values())

                distance_matrix = cdist(np.array(tracker_centroids) , input_centroids)

                rows = distance_matrix.min(axis=1).argsort()
                cols = distance_matrix.argmin(axis=1)[rows]

                usedRows = set()
                usedCols = set()
                
                tracker_ids = list(self.trackers.keys()) 
                for row,col in zip(rows,cols):
                    if row in usedRows or col in usedCols:
                        continue
                    track_id = tracker_ids[row]
                    
                    self.trackers[track_id] = input_centroids[col]
                    self.tracked_bboxes[track_id] = bboxes[col]

                    self.disappear_trackers[track_id] = 0
                    usedRows.add(row)                                
                    usedCols.add(col)
                unusedRows = set(range(0,distance_matrix.shape[0])).difference(usedRows)
                unusedCols = set(range(0,distance_matrix.shape[1])).difference(usedCols)
                if(distance_matrix.shape[0]>=distance_matrix.shape[1]):
                    
                    for r in unusedRows: 
                        track_id = tracker_ids[r]
                        self.disappear_trackers[track_id]+=1
                        if(self.disappear_trackers[track_id] > Tracker.MAX_DISAPPEAR_LIMIT):
                            self.del_object(track_id)
                else:
                    for c in unusedCols:                    
                        self.init_object(input_centroids[c],bboxes[c])

        return self.tracked_bboxes

## 3. Load Video

In [7]:
# Create tracker object
# tracker = EuclideanDistTracker()
tracker2 = Tracker()

cap = cv2.VideoCapture("c:/Users/Josep/Downloads/short_flow.mp4")
if not cap.isOpened():
    print("Error: No se puede abrir el archivo de video")

## 4. Object Detection and Tracking over video

In [ ]:
import warnings
warnings.filterwarnings("ignore")


previous_positions = {}  # Almacena la posición y anterior de cada coche
count_up = 0  # Conteo de coches que suben
count_down = 0  # Conteo de coches que bajan

# Umbral de movimiento para considerar si un coche se mueve lo suficiente
movement_threshold = 10

# Loop over video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Extract Region of interest
    roi = frame[350: 820, 100: 520]

    # 1. Object Detection
    results = model(roi)

    detections = []
    for det in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = det

        if model.names[int(cls)] == 'car':
            w = int(x2 - x1)
            h = int(y2 - y1)
            x = int(x1)
            y = int(y1)
            detections.append([x, y, w, h])

    # 2. Object Tracking
    # Update tracker
    boxes_ids = tracker2.update_object(detections)

    for box_id, box in boxes_ids.items():
        x, y, w, h = box
        id = box_id

        # Dibujar el ID y el rectángulo alrededor del coche detectado
        cv2.putText(roi, str(id), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Verificar si ya tenemos una posición anterior para este coche
        if id in previous_positions:
            prev_y = previous_positions[id]

            # Determinar si el coche sube o baja
            if abs(y - prev_y) > movement_threshold:  # El coche se mueve significativamente
                if y < prev_y:  # El coche está subiendo (disminuye su valor en y)
                    count_up += 1
                    print(f"Coche {id} sube. Total suben: {count_up}")
                elif y > prev_y:  # El coche está bajando (aumenta su valor en y)
                    count_down += 1
                    print(f"Coche {id} baja. Total bajan: {count_down}")

            # Actualizar la posición anterior del coche
            previous_positions[id] = y
        else:
            # Si es la primera vez que vemos este coche, registrar su posición
            previous_positions[id] = y

    # Mostrar las imágenes
    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)

    # 3. Control steps de frame
    cap.grab()

    key = cv2.waitKey(30)
    if key == 27:  # Escape key
        break

cap.release()
cv2.destroyAllWindows()

# Mostrar los conteos finales
print(f"Total coches que subieron: {count_up}")
print(f"Total coches que bajaron: {count_down}")


C:\Users\Josep/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Josep/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Josep/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Josep/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Josep/.cache\torch\hub\ultralytics_yolov5_master\models